In [49]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client["Ironhack"]
c = db.get_collection('I')
from bson.decimal128 import Decimal128
import re
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [4]:
#Developers like to be near successful tech startups that have raised at least 1 Million dollars.
#Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

In [47]:
pipeline = [
    {"$match": {"category_code": "games_video"}},
    {"$unwind": "$offices"},
    {"$group": {"_id": "$offices.country_code", "count": {"$sum": 1}}}
]

result = list(c.aggregate(pipeline))
result

[{'_id': 'DNK', 'count': 5},
 {'_id': 'HUN', 'count': 1},
 {'_id': 'USA', 'count': 587},
 {'_id': 'CHE', 'count': 6},
 {'_id': 'ESP', 'count': 14},
 {'_id': 'ARM', 'count': 1},
 {'_id': 'GBR', 'count': 74},
 {'_id': 'MEX', 'count': 4},
 {'_id': 'EST', 'count': 1},
 {'_id': 'AUS', 'count': 9},
 {'_id': 'CYP', 'count': 2},
 {'_id': 'ARA', 'count': 1},
 {'_id': 'SGP', 'count': 2},
 {'_id': 'COL', 'count': 1},
 {'_id': 'KOR', 'count': 3},
 {'_id': 'LBN', 'count': 1},
 {'_id': 'JPN', 'count': 14},
 {'_id': 'DEU', 'count': 30},
 {'_id': 'CHN', 'count': 17},
 {'_id': 'IND', 'count': 15},
 {'_id': 'IRL', 'count': 9},
 {'_id': 'CZE', 'count': 1},
 {'_id': 'UKR', 'count': 3},
 {'_id': 'NZL', 'count': 1},
 {'_id': 'CAN', 'count': 35},
 {'_id': 'ISL', 'count': 2},
 {'_id': 'IOT', 'count': 1},
 {'_id': 'PRT', 'count': 1},
 {'_id': 'SWE', 'count': 11},
 {'_id': 'SVN', 'count': 1},
 {'_id': 'MYS', 'count': 2},
 {'_id': 'TWN', 'count': 2},
 {'_id': 'ARG', 'count': 5},
 {'_id': 'GRC', 'count': 1},
 {'_

In [48]:
#GBR, ESP, FRA are 3 European countries with a decent amount of video game companies. Let's check if they also meet criteria for presence of successful start ups

In [58]:
condition1 = {"category_code": "games_video"} #vid game design
condition2 = {f"number_of_employees": {"$lt": 200}} #start-up characteristic
condition3 = {f"founded_year": {"$gte": 2010}} #start-up characteristic
regex_pattern = re.compile(r'\d+\.\d+M')
condition4 = {"total_money_raised": {"$regex": regex_pattern}}# Query condition to match companies with at least 1 million raised
condition5 = {f"offices.country_code": {"$in": ["ESP", "GBR", "FRA"]}}

In [60]:
pipeline = [
    {"$match": {"$and": [condition2, condition4]}},
    {"$unwind": "$offices"},
    {"$group": {"_id": "$offices.country_code", "count": {"$sum": 1}}}
    ]

result = list(c.aggregate(pipeline))
result

[{'_id': 'ISR', 'count': 31},
 {'_id': 'CHE', 'count': 4},
 {'_id': 'NLD', 'count': 12},
 {'_id': 'BGD', 'count': 1},
 {'_id': 'BEL', 'count': 7},
 {'_id': 'ROM', 'count': 4},
 {'_id': 'DNK', 'count': 8},
 {'_id': 'FRA', 'count': 32},
 {'_id': 'NOR', 'count': 3},
 {'_id': 'FIN', 'count': 4},
 {'_id': 'AUT', 'count': 1},
 {'_id': 'ITA', 'count': 4},
 {'_id': 'HKG', 'count': 4},
 {'_id': 'ARG', 'count': 3},
 {'_id': 'TWN', 'count': 1},
 {'_id': 'PHL', 'count': 2},
 {'_id': 'ANT', 'count': 1},
 {'_id': 'SWE', 'count': 9},
 {'_id': 'MYS', 'count': 2},
 {'_id': 'SVN', 'count': 1},
 {'_id': 'CAN', 'count': 29},
 {'_id': 'CYM', 'count': 1},
 {'_id': 'ISL', 'count': 1},
 {'_id': 'ARE', 'count': 1},
 {'_id': 'IRL', 'count': 8},
 {'_id': 'CZE', 'count': 2},
 {'_id': 'CHN', 'count': 8},
 {'_id': 'IND', 'count': 14},
 {'_id': 'KOR', 'count': 3},
 {'_id': 'DEU', 'count': 20},
 {'_id': 'JPN', 'count': 8},
 {'_id': 'SGP', 'count': 7},
 {'_id': 'PRK', 'count': 1},
 {'_id': 'COL', 'count': 1},
 {'_id':

In [ ]:
#GBR, FRA, ESP all good again for small companies with total raised above $1M.

In [43]:
#Looking in Europe because having bars, public space/daycare close by for kids etc is much easier in Europe in my opinion